# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

**Note:** The versions of crewai mentioned above are outdated and don't support all functionalities and use-cases of crewai. I instead used the latest verions of crewai by not specifying a version in the installation command.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai crewai_tools langchain_community
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew, LLM

In [3]:
import os
from utils import get_openai_api_key
from langchain_openai import ChatOpenAI

In [4]:
# os.environ["OPENAI_API_KEY"] = "sk-proj-0000"
# os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
# os.environ["OPENAI_MODEL_NAME"] = "llama3.1:latest"

os.environ["OPENAI_API_KEY"] = "sk-xxxxx"  # Set a dummy key if needed

custom_llm = LLM(
    # model="ollama/llama3.1:latest", 
    model="mistral:latest",
    base_url="http://localhost:11434"
)


## Role Playing, Focus and Cooperation

# Creating Agents

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
	llm = LLM(
		# model="ollama/llama3.1:latest", 
		model="ollama/mistral:latest",
		base_url="http://localhost:11434"
	)
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm = LLM(
		# model="ollama/llama3.1:latest", 
		model="ollama/mistral:latest",
		base_url="http://localhost:11434/v1"
	)
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a answer to a customer query to align with the writing style of "
         " the organization and ensure that it has a professional and friendly tone throughout.. ",
    backstory="You are an editor who receives an answer to a customer's query or question "
              "from the Support Quality Assurance Specialist. "
              "Your goal is to review the reponse message and make sure that it "
              "follows best practices for professional business dialogs, "
              "and make sure that it is easy to understand and simple to follow.",
    allow_delegation=False,
    verbose=True,
    # llm=custom_llm
    llm = LLM(
		model="ollama/llama3.1:latest", 
		base_url="http://localhost:11434"
	)
)

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [8]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/crewai_tools/tools/selenium_scraping_to

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [9]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/crews"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [10]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [11]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [12]:
edit = Task(
    description=("Proofread the reponse message for "
                 "grammatical errors and "
                 "alignment with best professional practices."),
    expected_output=("A well-written response message in markdown format, "
                    "ready for sending to the customer."
                ),
    agent=editor
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [13]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

2025-02-22 17:05:29,759 - 133128381269568 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/http/client.py", line 1395, in getresponse
    response.begin()
  File "/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/http/client.py", line 286, in _read_sta

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [14]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

2025-02-22 16:49:11,584 - 133130105554752 - rag_storage.py-rag_storage:138 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-02-22 16:49:13,223 - 133130105554752 - rag_storage.py-rag_storage:138 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Using tool: Read website content
## Tool Input: 
"{\"url\": \"https://docs.crewai.com/concepts/crews\"}"
## Tool Output: 
Crews - CrewAI CrewAI home page Search CrewAI docs crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Core Concepts Crews Get Started Examples CrewAI home page Community Changelog Get Started Introduction Installation Quickstart Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration Training Memory Planning Testing CLI Tools Using LangChain Tool

2025-02-22 17:02:34,770 - 133130105554752 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
/home/mohammed/anaconda3/envs/multiagent/lib/python3.11/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
2025-02-22 17:04:54,062 - 133130105554752 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-02-22 17:04:54,505 - 133130105554752 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-02-22 17:04:54,985 - 133130105554752 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only argumen

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.APIConnectionError: OllamaException - 404 page not found


APIConnectionError: litellm.APIConnectionError: OllamaException - 404 page not found

- Display the final result as Markdown.

In [ ]:
clean_output = result.raw
print(clean_output)

In [ ]:
from IPython.display import Markdown
Markdown(clean_output)